## Prerequisites

Note: Skip if you already have the prerequisites installed.

### SyftUI
Make sure you have SyftUI running with your email logged in.
1. It could be downloaded from [SyftUI](https://github.com/OpenMined/SyftUI/releases)


### Install dependencies for this demo

### Python Environment

Create a Virutal Environment
```bash
uv venv --python 3.12
source .venv/bin/activate
```

### Install Dependencies

```bash
uv pip install jupyterlab syftbox-enclave syft-rds==0.1.1-dev.2
```

## Explore Datasets

In [ ]:
from syft_rds import init_session
from syftbox_enclave.client import connect

In [ ]:
DOS = ["<DO1@EMAIL>", "<DO2@EMAIL>"]
DS = "<DS@EMAIL>"
ENCLAVE = "enclave-organic-coop@openmined.org"


In [ ]:
# Get a client connection to the Data Owners (DOs)
datasites = []
for datasite_email in DOS:
    datasite_client = init_session(host=datasite_email)
    print("Logged into: ", datasite_client.host)
    datasites.append(datasite_client)

In [ ]:
# List Datasets of a data owner
datasites[0].datasets

In [ ]:
# Retrieve a specific dataset from each Data Owner
DATASET_NAME = "<DATASET_NAME>"

datasets = []
for datasite_client in datasites:
    dataset = datasite_client.datasets.get(name=DATASET_NAME)
    datasets.append(dataset)
    print(f"Dataset '{DATASET_NAME}' from {datasite_client.host} retrieved.")

## Prototype Code

In [ ]:
from pathlib import Path

code_path = Path(".") / "code"
code_path.mkdir(exist_ok=True)

code_file = code_path / "entrypoint.py"

In [ ]:
%%writefile {code_file}

import os
from pathlib import Path
from sys import exit

import pandas as pd

DATA_DIR = os.environ["DATA_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]

dataset_paths = [ Path(dataset_path) for dataset_path in DATA_DIR.split(",")]
total_carrots = 0
total_tomatoes = 0

for dataset_path in dataset_paths:
    if not dataset_path.exists():
        print("Warning: Dataset path does not exist:", dataset_path)
        exit(1)
    df = pd.read_csv(dataset_path / "crop_stock_data.csv")
    total_carrots += df[df["Product name"] == "Carrots"]["Quantity"].sum()
    total_tomatoes += df[df["Product name"] == "Tomatoes"]["Quantity"].sum()

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    f.write(f"Total Carrots: {total_carrots}\n")
    f.write(f"Total Tomatoes: {total_tomatoes}\n")

### DO Job

In [ ]:
# Job submission

for datasite_client in datasites:
    job = datasite_client.jobs.submit(
            name="Organic Crop Stock Analysis",
            description="Farming Coop Avg Experiment",
            user_code_path="./code",
            dataset_name=DATASET_NAME,
            tags=["enclave", "syft"],
            entrypoint="entrypoint.py",
            enclave = ENCLAVE
        )
    print(job)

In [ ]:
datasites[0].jobs.get_all()

### Enclave Client

In [ ]:
enclave_client = connect(ENCLAVE)

In [ ]:
proj_res = enclave_client.create_project(
                   project_name = "<PROJECT_NAME>",
                   datasets = datasets,
                   output_owners = [*DOS,DS],
                   code_path = "./code",
                   entrypoint = "entrypoint.py"
            )
            

In [ ]:
# Wait until the project is ready
proj_res.output()